# Process Load Data for the GODEEEP Project

This notebook merges together the time series of hourly electricity demand from transportation and non-transportation sources by Balancing Authority (BA) and produces the output in a format ready to ingest into GridView.

In [1]:
# Start by importing the packages we need:
import os
import glob
import warnings
import datetime

import pandas as pd
import numpy as np


## Set the Directory Structure

In [2]:
# Identify the data input and output directories:
trans_data_input_dir =  '/Users/burl878/Documents/GODEEEP/Data/Transportation/Raw/'
tell_data_input_dir =  '/Users/burl878/Documents/GODEEEP/Data/TELL/Production_Runs/tell_data/outputs/tell_output/'
merged_data_output_dir =  '/Users/burl878/Documents/GODEEEP/Data/Merged_BA_Loads/'


## Set the Scenario and Year You Want to Process

In [3]:
gcam_scenario_to_process = 'BAU_Climate'
year_to_process = '2025'


## Merge the Transportation Output Files Together

In [4]:
def merge_transportation_data(gcam_scenario_to_process: str, year_to_process: str, trans_data_input_dir: str):
   
    # Create a list of all of the transportation output files in the "trans_data_input_dir" and aggregate the files in that list:
    trans_filelist = sorted(glob.glob(os.path.join(trans_data_input_dir, gcam_scenario_to_process, 'rcp85hotter', ('*' + year_to_process + '.csv'))))
       
    # Loop over the list of files:
    for file in range(len(trans_filelist)):
        # Read in the .csv file:
        trn_data = pd.read_csv(trans_filelist[file])

        # Rename a few variables for consistency with TELL:
        trn_data.rename(columns={'balancing_authority': 'BA_Code',
                                 'time': 'Time_UTC',
                                 'transportation_load_MWh': 'Transportation_Load_MWh'}, inplace=True)
           
        # Shift the timestampe by a single hour:
        trn_data['Time_UTC'] = (pd.to_datetime(trn_data['Time_UTC']) - pd.DateOffset(hours=1)).dt.strftime("%Y-%m-%d %H:%M:%S")

        # Only keep the columns that are needed:
        trn_data = trn_data[['BA_Code', 'Time_UTC', 'Transportation_Load_MWh']].copy()
        
        # Strip the "+00:00" from the time string:
        trn_data['Time_UTC'] = trn_data['Time_UTC'].astype(str)
        trn_data['Time_UTC'] = trn_data['Time_UTC'].str.split('+').str[0]

        # Aggregate the output into a new dataframe:
        if file == 0:
           trn_output_df = trn_data
        else:
           trn_output_df = pd.concat([trn_output_df, trn_data])
      
        # Clean up and move to the next file:
        del trn_data
           
    return trn_output_df


In [5]:
# Merge the transportation output files into a single dataframe:
trn_df = merge_transportation_data(gcam_scenario_to_process = gcam_scenario_to_process, 
                                   year_to_process = year_to_process, 
                                   trans_data_input_dir = trans_data_input_dir)

# Preview the transportation dataframe:
trn_df


,BA_Code,Time_UTC,Transportation_Load_MWh
0,AVA,2025-01-01 00:00:00,48.821954
1,AVA,2025-01-01 01:00:00,56.373702
2,AVA,2025-01-01 02:00:00,51.710269
3,AVA,2025-01-01 03:00:00,44.191415
4,AVA,2025-01-01 04:00:00,38.775120
...,...,...,...
8755,WWA,2025-12-31 19:00:00,0.448920
8756,WWA,2025-12-31 20:00:00,0.569309
8757,WWA,2025-12-31 21:00:00,0.571928
8758,WWA,2025-12-31 22:00:00,0.704104


## Read in the TELL Output File and Subset to Just the BAs in the WECC


In [6]:
# Read in the BA-to-Interconnection mapping file:
ba_mapping_df = pd.read_csv(merged_data_output_dir + 'BA_to_Interconnection_Mapping.csv')


In [7]:
# Read in the TELL BA output file for the year and scenario being processed:
tell_df = pd.read_csv(tell_data_input_dir + gcam_scenario_to_process + '/' + year_to_process + '/TELL_Balancing_Authority_Hourly_Load_Data_' + year_to_process + '_Scaled_' + year_to_process + '.csv')

# Rename a few variables for consistency:
tell_df.rename(columns={'Scaled_TELL_BA_Load_MWh': 'Non-Transportation_Load_MWh'}, inplace=True)

# Merge the ba_mapping_df and tell_df dataframes based on common "BA_Code":
tell_df = tell_df.merge(ba_mapping_df, on=['BA_Code'])

# Subset to just the WECC BAs:
tell_df = tell_df[tell_df['Interconnection'] == 'WECC'].copy()

# Make the time variable a string:
tell_df['Time_UTC'] = tell_df['Time_UTC'].astype(str)

# Only keep the columns that are needed:
tell_df = tell_df[['BA_Code', 'Time_UTC', 'Non-Transportation_Load_MWh']].copy()

# Preview the TELL dataframe:
tell_df


,BA_Code,Time_UTC,Non-Transportation_Load_MWh
17520,AVA,2025-01-01 00:00:00,2179.79
17521,AVA,2025-01-01 01:00:00,2264.92
17522,AVA,2025-01-01 02:00:00,2336.19
17523,AVA,2025-01-01 03:00:00,2383.70
17524,AVA,2025-01-01 04:00:00,2338.14
...,...,...,...
473035,WAUW,2025-12-31 19:00:00,90.18
473036,WAUW,2025-12-31 20:00:00,90.02
473037,WAUW,2025-12-31 21:00:00,90.12
473038,WAUW,2025-12-31 22:00:00,90.83


## Merge the TELL and Transportation Dataframes Together


In [8]:
# Merge the trn_df and tell_df dataframes based on common "BA_Code" and "Time_UTC":
merged_df = tell_df.merge(trn_df, on=['BA_Code', 'Time_UTC'])

# Compute the sum of the transportation and non-transportation loads:
merged_df['Total_Load_MWh'] = merged_df['Non-Transportation_Load_MWh'] + merged_df['Transportation_Load_MWh']

# Round off the values to make the output file more readable:
merged_df['Non-Transportation_Load_MWh'] = merged_df['Non-Transportation_Load_MWh'].round(2)
merged_df['Transportation_Load_MWh'] = merged_df['Transportation_Load_MWh'].round(2)
merged_df['Total_Load_MWh'] = merged_df['Total_Load_MWh'].round(2)

# Fill in missing values with -9999:
merged_df['Non-Transportation_Load_MWh'] = merged_df['Non-Transportation_Load_MWh'].fillna(-9999)
merged_df['Transportation_Load_MWh'] = merged_df['Transportation_Load_MWh'].fillna(-9999)
merged_df['Total_Load_MWh'] = merged_df['Total_Load_MWh'].fillna(-9999)

# Rename the BA variable:
merged_df.rename(columns={'BA_Code': 'BA'}, inplace=True) 

# Write out the dataframe to a .csv file:
merged_df.to_csv((os.path.join(merged_data_output_dir, ('Total_Loads_' + gcam_scenario_to_process + '_' + year_to_process + '.csv'))), sep=',', index=False)

# Preview the merged dataframe:
merged_df


,BA,Time_UTC,Non-Transportation_Load_MWh,Transportation_Load_MWh,Total_Load_MWh
0,AVA,2025-01-01 00:00:00,2179.79,48.82,2228.61
1,AVA,2025-01-01 01:00:00,2264.92,56.37,2321.29
2,AVA,2025-01-01 02:00:00,2336.19,51.71,2387.90
3,AVA,2025-01-01 03:00:00,2383.70,44.19,2427.89
4,AVA,2025-01-01 04:00:00,2338.14,38.78,2376.92
...,...,...,...,...,...
245275,WAUW,2025-12-31 19:00:00,90.18,10.18,100.36
245276,WAUW,2025-12-31 20:00:00,90.02,10.31,100.33
245277,WAUW,2025-12-31 21:00:00,90.12,13.70,103.82
245278,WAUW,2025-12-31 22:00:00,90.83,15.81,106.64


## Format the Data for Ingest into GridView


In [9]:
def process_gridview_data(merged_data_output_dir: str):
    
    # Read in the raw data .csv file:
    gv_df = pd.read_csv((merged_data_output_dir + '2030_Load.csv'))
    
    # Subset to just the annual total demand by BA:
    gv_df = gv_df[-3:-2]
       
    # Strip the unecessary bits from the column names:
    gv_df.columns = gv_df.columns.str.replace("_CEC", "", regex=True)
    gv_df.columns = gv_df.columns.str.replace("_2030.dat", "", regex=True)
    gv_df.columns = gv_df.columns.str.replace("Load_", "", regex=True)
       
    # Delete the index and last column:
    del gv_df["Index"], gv_df["Unnamed: 44"]
    
    # Convert the values to floats:
    gv_df = gv_df.astype('float64')
    
    # Compute the total loads for CISO, IPCO, NEVP, and PACE:
    gv_df['CISO'] = (gv_df['CIPB'] + gv_df['CIPV'] + gv_df['CISC'] + gv_df['CISD'] + gv_df['VEA']).round(2)
    gv_df['IPCO'] = (gv_df['IPFE'] + gv_df['IPMV'] + gv_df['IPTV']).round(2)
    gv_df['PACE'] = (gv_df['PAID'] + gv_df['PAUT'] + gv_df['PAWY']).round(2)
    gv_df['NEVP_Sum'] = (gv_df['NEVP'] + gv_df['SPPC']).round(2)
           
    # Rename a few columns for consistency:
    gv_df.rename(columns={'CIPB': 'CISO_CIPB', 'CIPV': 'CISO_CIPV', 'CISC': 'CISO_CISC', 'CISD': 'CISO_CISD', 'VEA': 'CISO_VEA',
                          'IPFE': 'IPCO_IPFE', 'IPMV': 'IPCO_IPMV', 'IPTV': 'IPCO_IPTV',
                          'NEVP': 'NEVP_NEVP', 'SPPC': 'NEVP_SPPC',
                          'PAID': 'PACE_PAID', 'PAUT': 'PACE_PAUT', 'PAWY': 'PACE_PAWY'}, inplace=True) 
    gv_df.rename(columns={'NEVP_Sum': 'NEVP'}, inplace=True) 
    
    # Squeeze the dataframe:
    gv_df = gv_df.squeeze().to_frame()
        
    # Rename the columns:
    gv_df.reset_index(inplace=True)
    gv_df = gv_df.rename(columns = {'index':'BA'})
    gv_df.rename(columns={gv_df.columns[1]: "Total_Load_MWh" }, inplace = True)
       
    # Sort the dataframe alphabetically by BA name:
    gv_df = gv_df.sort_values('BA')
       
    # Return the output dataframe:
    return gv_df


In [10]:
def format_loads_for_gridview(merged_data_output_dir: str):
    
    # Process the GridView file:
    gv_df = process_gridview_data(merged_data_output_dir = merged_data_output_dir)
    
    # Compute the load fractions for the subregions:
    CIPB_LF = (gv_df.loc[(gv_df['BA'] == 'CISO_CIPB')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['Total_Load_MWh'].values[0])
    CIPV_LF = (gv_df.loc[(gv_df['BA'] == 'CISO_CIPV')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['Total_Load_MWh'].values[0])
    CISC_LF = (gv_df.loc[(gv_df['BA'] == 'CISO_CISC')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['Total_Load_MWh'].values[0])
    CISD_LF = (gv_df.loc[(gv_df['BA'] == 'CISO_CISD')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['Total_Load_MWh'].values[0])
    VEA_LF  = (gv_df.loc[(gv_df['BA'] == 'CISO_VEA' )]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'CISO')]['Total_Load_MWh'].values[0])
    IPFE_LF = (gv_df.loc[(gv_df['BA'] == 'IPCO_IPFE')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'IPCO')]['Total_Load_MWh'].values[0])
    IPMV_LF = (gv_df.loc[(gv_df['BA'] == 'IPCO_IPMV')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'IPCO')]['Total_Load_MWh'].values[0])
    IPTV_LF = (gv_df.loc[(gv_df['BA'] == 'IPCO_IPTV')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'IPCO')]['Total_Load_MWh'].values[0])
    NEVP_LF = (gv_df.loc[(gv_df['BA'] == 'NEVP_NEVP')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'NEVP')]['Total_Load_MWh'].values[0])
    SPPC_LF = (gv_df.loc[(gv_df['BA'] == 'NEVP_SPPC')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'NEVP')]['Total_Load_MWh'].values[0])
    PAID_LF = (gv_df.loc[(gv_df['BA'] == 'PACE_PAID')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PACE')]['Total_Load_MWh'].values[0])
    PAUT_LF = (gv_df.loc[(gv_df['BA'] == 'PACE_PAUT')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PACE')]['Total_Load_MWh'].values[0])
    PAWY_LF = (gv_df.loc[(gv_df['BA'] == 'PACE_PAWY')]['Total_Load_MWh'].values[0]) / (gv_df.loc[(gv_df['BA'] == 'PACE')]['Total_Load_MWh'].values[0])
    
    # Read in the merged total load file:
    tell_df = pd.read_csv((os.path.join(merged_data_output_dir, ('Total_Loads_' + gcam_scenario_to_process + '_' + year_to_process + '.csv'))))
     
    # Compute the hours since the start of the year:
    tell_df['Hour'] = ((pd.to_datetime(tell_df['Time_UTC']) - datetime.datetime(int(year_to_process), 1, 1, 0, 0, 0)) / np.timedelta64(1, 'h') + 1).astype(int)
    
    # Reshape the dataframe and reset the indexes:
    load_df = tell_df.pivot(index = 'Hour', columns = 'BA', values = 'Total_Load_MWh')
    load_df = load_df.reset_index(drop=False)
    
    # Add back in the text to the column headers:
    load_df = load_df.add_suffix('_2030.dat')
    load_df = load_df.add_prefix('Load_')
    
    # Rename the time variable:
    load_df.rename(columns={'Load_Hour_2030.dat': 'Index'}, inplace=True)
    
    # Compute the loads for the subregions:
    load_df['Load_CIPB_2030_CEC.dat'] = load_df['Load_CISO_2030.dat'] * CIPB_LF
    load_df['Load_CIPV_2030_CEC.dat'] = load_df['Load_CISO_2030.dat'] * CIPV_LF
    load_df['Load_CISC_2030_CEC.dat'] = load_df['Load_CISO_2030.dat'] * CISC_LF
    load_df['Load_CISD_2030_CEC.dat'] = load_df['Load_CISO_2030.dat'] * CISD_LF
    load_df['Load_VEA_2030.dat'] = load_df['Load_CISO_2030.dat'] * VEA_LF
    load_df['Load_IPFE_2030.dat'] = load_df['Load_IPCO_2030.dat'] * IPFE_LF
    load_df['Load_IPMV_2030.dat'] = load_df['Load_IPCO_2030.dat'] * IPMV_LF
    load_df['Load_IPTV_2030.dat'] = load_df['Load_IPCO_2030.dat'] * IPTV_LF
    load_df['Load_NEVP_Temp_2030.dat'] = load_df['Load_NEVP_2030.dat'] * NEVP_LF
    load_df['Load_SPPC_2030.dat'] = load_df['Load_NEVP_2030.dat'] * SPPC_LF
    load_df['Load_PAID_2030.dat'] = load_df['Load_PACE_2030.dat'] * PAID_LF
    load_df['Load_PAUT_2030.dat'] = load_df['Load_PACE_2030.dat'] * PAUT_LF
    load_df['Load_PAWY_2030.dat'] = load_df['Load_PACE_2030.dat'] * PAWY_LF
    
    # Drop the un-needed columns for BAs with subregions:
    del load_df['Load_NEVP_2030.dat'], load_df['Load_CISO_2030.dat'], load_df['Load_IPCO_2030.dat'], load_df['Load_PACE_2030.dat']
    
    # Clean up the NEVP naming:
    load_df.rename(columns={'Load_NEVP_Temp_2030.dat': 'Load_NEVP_2030.dat'}, inplace=True)
    
    # Read in the raw data GridView .csv file:
    raw_gv_df = pd.read_csv((merged_data_output_dir + '2030_Load.csv'))
    
    # Delete the index column:
    del raw_gv_df["Index"] 
    
    # Subset to just the rows we need:
    raw_gv_df = raw_gv_df[1:8761]
    
    # Convert the values to floats:
    raw_gv_df = raw_gv_df.astype('float64')
    
    # Shift the index by -1:
    raw_gv_df = raw_gv_df.reset_index()
    
    # Merge in the GridView columns that aren't modeled by TELL:
    load_df = pd.concat([load_df,raw_gv_df['Load_AESO_2030.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_BCHA_2030.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_CFE_2030.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_TH_Malin_2030.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_TH_Mead_2030.dat']], axis=1)
    load_df = pd.concat([load_df,raw_gv_df['Load_TH_PV_2030.dat']], axis=1)
    
    # Compute the summary statistics:
    stats_df = load_df.apply(['mean','sum','max','min'])
    
    # Fix the summary statistic labels:
    stats_df.iloc[0, 0] = 'AVG'
    stats_df.iloc[1, 0] = 'SUM'
    stats_df.iloc[2, 0] = 'MAX'
    stats_df.iloc[3, 0] = 'MIN'
    
    # Create a target dataframe with the spare hours:
    target_df = pd.DataFrame({"Index": np.arange(1,8791,1)})
    
    # Merge load dataframe with the target dataframe:
    merged_df = target_df.merge(load_df, on=['Index'], how='left')
    
    # Sort the data by column name and make the Index column appear first:
    merged_df.rename(columns={'Index': 'AA'}, inplace=True)
    merged_df = merged_df.sort_index(axis = 1)
    merged_df.rename(columns={'AA': 'Index'}, inplace=True)
    
    # Add in a blank row and fill it with the year placeholder:
    merged_df.loc[-0.5] = 0
    merged_df = merged_df.sort_index().reset_index(drop=True)
    merged_df.iloc[0, :] = '2030'
    merged_df.at[0, 'Index'] = 'Year'
    
    # Merge the hourly load data and statistics dataframes together:
    output_df = pd.concat([merged_df, stats_df], axis=0)
    
    # Replace NaNs with blank values:
    output_df.replace(np.nan, "", regex=True)
    
    # Round all the numerical values to two decimals
    output_df.round(2)
    
    # Write out the dataframe to a .csv file:
    output_df.to_csv((os.path.join(merged_data_output_dir, ('GridView_Loads_' + gcam_scenario_to_process + '_' + year_to_process + '.csv'))), sep=',', index=False)
    
    # Return the output dataframe:
    return output_df


In [11]:
gridview_df = format_loads_for_gridview(merged_data_output_dir = merged_data_output_dir)

gridview_df


,Index,Load_AESO_2030.dat,Load_AVA_2030.dat,Load_AZPS_2030.dat,Load_BANC_2030.dat,Load_BCHA_2030.dat,Load_BPAT_2030.dat,Load_CFE_2030.dat,Load_CHPD_2030.dat,Load_CIPB_2030_CEC.dat,...,Load_TEPC_2030.dat,Load_TH_Malin_2030.dat,Load_TH_Mead_2030.dat,Load_TH_PV_2030.dat,Load_TIDC_2030.dat,Load_TPWR_2030.dat,Load_VEA_2030.dat,Load_WACM_2030.dat,Load_WALC_2030.dat,Load_WAUW_2030.dat
0,Year,2030,2030,2030,2030,2030,2030,2030,2030,2030,...,2030,2030,2030,2030,2030,2030,2030,2030,2030,2030
1,1,10961.0,2228.61,3465.58,2165.62,8020.0,8990.03,1775.0,415.61,5552.325214,...,1778.44,0.0,0.0,0.0,285.85,954.02,85.295016,3909.8,1009.88,140.81
2,2,10826.0,2321.29,3722.96,2413.03,7440.0,9670.21,1707.0,424.93,6015.639151,...,1878.92,0.0,0.0,0.0,328.28,1016.97,92.412461,3922.7,1071.49,138.23
3,3,10744.0,2387.9,3770.27,2515.69,7173.0,10058.16,1665.0,420.04,6167.484021,...,1876.06,0.0,0.0,0.0,334.07,1046.5,94.745107,3949.95,1085.37,133.83
4,4,10738.0,2427.89,3799.24,2626.13,7027.0,10298.52,1639.0,412.75,6268.349031,...,1864.14,0.0,0.0,0.0,336.79,1060.75,96.294599,3912.22,1088.64,131.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8790,8790,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,AVG,10997.146918,1674.956724,4112.74029,2445.48101,7497.783904,7657.292586,2514.986416,260.814889,6326.256259,...,1810.495652,0.0,0.0,0.0,378.161355,708.636263,97.184172,3531.84792,1219.872777,100.580944
sum,SUM,96335007.0,14672620.9,36027604.94,21422413.65,65680587.0,67077883.05,22031281.0,2284738.43,55418004.82976,...,15859941.91,0.0,0.0,0.0,3312693.47,6207653.66,851333.345624,30938987.78,10686085.53,881089.07
max,MAX,13241.0,2751.21,8787.15,5507.06,12204.0,12225.94,4301.0,546.13,12110.136725,...,3725.02,0.0,0.0,0.0,786.09,1168.0,186.036348,5205.68,2006.87,167.97
